## Myanmar NER 
- NER has been carried out on all Peace Agreements from Myanmar
- Results of model pick up incorrect labels for entities, or else not the whole names ==> manual dictionaries are required to fix this
- These dictionaries also allow us to link aliases for the same entities to be labelled as the same 
- This has been carried out on people and organisations so far, but will do from scratch again for a master dataset
- Also can use the dictionary from ICEWS actors, so this will now be included

### First - read in the results from the NER model:
- The model recognised 1,377 entities in all the agreements which is useful 
- But can see the issues with these results straight away below;
- 'CNF - Govt Agreements' and 'Chin National (CNF' both mean the same thing, recognised in the same agreement but there is no way of knowing that they mean the same from these results

In [1]:
import pandas as pd

In [2]:
results=pd.read_csv('all_entities_myanmar.csv').drop(columns=['Unnamed: 0'])

In [3]:
results

,doc_id,agreement_id,sentence_number,entity,entity_type
0,0,1321,0,CNF - Govt Agreements,ORG
1,0,1321,0,Chinland Guardian,ORG
2,0,1321,0,Chin National ( CNF,ORG
3,0,1321,0,Peace Committee Burma,ORG
4,0,1321,0,Rangoon,GPE
...,...,...,...,...,...
1372,35,776,4,Rakhine State Peace Discussion Group,ORG
1373,35,776,4,State Security,ORG
1374,35,776,4,Htein Lin,PERSON
1375,35,776,5,Rakhine State Liberation Party,ORG


## Dictionaries
- One way to address this issue is through manual dictionaries, which have been developed
- Have a named entity -> actual entity dictionary 
- And a person -> affiliation as it is useful to know who the individual people are representing 

First, useful to correct all organisation names, so we know what organisations are mentioned - this dict helps recognise aliases or abbreviations. 

In [4]:
org_dict={
    'Chin National Front (CNF)':[' CNF - Govt Agreements',' Chin National ( CNF', ' CNF', ' Chin National Chin State ', 'Chin National / Chin National Army',  ' Chin', ' Chin National Day', ' Chin National Peace Team', ' CNA',  ' Chin National ’s', ' CNF Government',  ' Chin National / Army', ' Chin National Union', ' Chin National Chin State / Union Government', ' Chin National Peace Negotiating Team', ' Chin State Government', ' Government Chin State', ' Chin Special Division', ' Chin State Union Government', ' Chin National Peace Team', ' Chin National Chin State / Union Government', ' Chin National', ' Chin National Chin National Army', ' Chin National Army'],
    'Joint Monitoring Committee (JMC)':[' Joint Monitoring Committee', ' JMC', ' Joint Monitoring Committee Guideline'],
    'Joint Ceasefire Monitoring Committee (JCMC)': [ ' Joint Ceasefire Monitoring Committee', ' Ceasefire Joint Monitoring Committee', ' Union Level Joint Ceasefire Monitoring Committee', ' Local Level Joint Ceasefire Monitoring Committee',' Nationwide Ceasefire Agreement Joint Implementation Coordination Meeting'],
    'Union Level Joint Ceasefire Monitoring Committee (JMC-U)':[' JMC - U', ' JMC - U.  Meetings Resolutions', ' Union Peace Dialogue Joint Committee'],
    'State Level Joint Ceasefire Monitoring Committee (JMC-S)':[' JMC - S', ' JMC - S.', ' JMC - S. '],
    'Local Level Joint Ceasefire Monitoring Committee (JMC-L)':[' JMC - L', ' JMC - L.  Responsibilities JMC - L'],
    'Karen National Union (KNU)/Karen National Liberation Army (KNLA)':['Karen National Union', ' KNLA  Permanent Central Committee', ' KNU', ' KNU / KNLA', ' KNU Press Release', ' KNU Delegation', ' Democratic Karen Benevolent Army',' Karen National Union Foreign & Public Relations Department', ' KNU / KNLA Peace Council 9',  ' KNLA Peace Council', ' KNU / KNLA Peace Council', ' Democratic Karen Benevolent Army'],
    'Karenni National Progressive Party (KNPP)':[' Karenni National Progressive Party', ' KNPP', ' KNPP Khu Oo Reh'],
    'Union Peace Dialogue Joint Committee (UPDJC)':[' UPDJC', ' Union Peace Working Committee', ' Union Peace - Making Work Committee', ' Union Peace Dialogue Joint Committee', ' Union Peace',  ' Union Peace Team Crisis Mediating Body'],
    'Government of the Republic of the Union of Myanmar':[' Union Government', ' Government Republic Union Myanmar', ' Government ’s Order',' Government Group', ' Myanmar Government', ' Government Burma',' Burmese Government',],
    'Hluttaw':[' Hluttaw', ' Hluttaws', ' Hluttaw Group  3', ' Pyidaungsu Hluttaw'],
    'Tatmadaw':[' Tatmadaw', ' Pyidaungsu Tatmadaw', ' Tatmadaw Group  4'],
    'Shan State Progress Party (SSPP)/Shan State Army (SSA)':[' SSA - S Government Ceasefire Agreement',' SSA - S Government 11',' SSA', ' SSA - S',' Restoration Council Shan State / Shan State Army',' SSPP / SSA - N Government  Preliminary Peace', ' Shan State Progressive Party', ' SSPP)/ Shan State Army',' Shan State',' SSA - S Government',  ' Shan State Minster Security', ' SSPP)/ Shan State Army', ' Shan State',' SSPP / Shan State Army Sao Khun Hsai',  ' SSPP / Shan State Army', ' Shan State Progressive',  ' Party/ Shan State Army', ' Shan State Police Force', ' Mongla Armed Group ( Special Region 4 - Northern Shan State',' Shan State Progressive Party / Shan State Army',],
    'Arakan Army':[ ' Arakan Liberation Party Arakan National Council', ' General Arakan Army Chin National'],
    'Rakhine State Liberation Party (RSLP)':[ ' Rakhine State Liberation Party', ],
    'Ethnic Armed Organizations (EAO)': [ ' Ethnic Armed Organizations', ' Ethnic Armed  Organizations',' Ethnic Armed Organization', ' Ethnic Armed Organization Group  5', ' Ethnic Armed Organization Group  U Zein Khan Ram  Ethnic Representative Political Parties Group', ' ee ) EAOs ',],
    'Food and Drug Administration (FDA)': [' FDA'], 
    'National League Democracy (NLD)':[' NLD'],
    'United Nationalities Alliance (UNA)':[' UNA',],
    'Lahu Democratic Union':[ ' Lahu Democratic Union Myanmar National Democratic Alliance Army',],
    'United Nations':[' U.N.'],
    'Pa-O National Liberation Organisation (PNLO)': [' PNLO'],
    'Democratic Karen Benevolent/Buddhist Army (DKBA)':[ ' Democratic Karen Benevolent Army',],
    'United Nations Office on Drugs and Crime (UNODC)':[' UNODC'],
    'Union Peacemaking Working Committee (UPWC)':[' UPWC'],
    'New Mon State Party (NMSP)': [' NMSP'],
    'Union Election Commission (UEC)': [' UEC'],
    'Kachin Independence Organisation (KIO)': [' KIO',' Kachin Independence Organization'],
    'United Nationalities Federal Council (UNFC)': [' UNFC'],
    'National Socialist Council of Nagaland (NSCN)':[' Naga', ' NSCN'],
    'All Burma Students Democratic Front (ABSDF)': [' ABSDF Union', ' ABSDF'],
    'Nationalities Brotherhood Federation (NBF)': [' NBF']
    
    
}

To effectively use this data in visualisations later, the full organisation name is usually too long, so an organisation abbreviation dictionary was made:

In [5]:
shortened_name_dict={
    'CNF': 'Chin National Front (CNF)',
    'JMC':  'Joint Monitoring Committee (JMC)',
    'JICM':'Joint Implementing Coordination Meeting (JICM)',
    'JCMC':'Joint Ceasefire Monitoring Committee (JCMC)',
    'JMC-U':'Union Level Joint Ceasefire Monitoring Committee (JMC-U)',
    'JMC-S':'State Level Joint Ceasefire Monitoring Committee (JMC-S)',
    'JMC-L':'Local Level Joint Ceasefire Monitoring Committee (JMC-L)',
    'KNU/KNLA':'Karen National Union (KNU)/Karen National Liberation Army (KNLA)',
    'KNPP':'Karenni National Progressive Party (KNPP)',
    'UPDJC':'Union Peace Dialogue Joint Committee (UPDJC)',
    'Gov. of Myanmar':'Government of the Republic of the Union of Myanmar',
    'Hluttaw':'Hluttaw',
    'Tatmadaw':'Tatmadaw',
    'SSPP/SSA':['Shan State Progress Party (SSPP)/Shan State Army (SSA)', 'SSPP', 'SSPP/SSA'],
    'AA':'Arakan Army',
    'RSLP':'Rakhine State Liberation Party (RSLP)',
    'EAO':'Ethnic Armed Organizations (EAO)',
    'FDA':'Food and Drug Administration (FDA)',
    'NLD':'National League Democracy (NLD)',
    'UNA':'United Nationalities Alliance (UNA)',
    'LDU':'Lahu Democratic Union',
    'UN':'United Nations',
    'DKBA':'Democratic Karen Benevolent/Buddhist Army (DKBA)',
    'ABSDF':'All Burma Students Democratic Front (ABSDF)',
    'RCSS':'Restoration Council of Shan State (RCSS)',
    'DKBA-5':'Kalo Htoo Baw faction of Democratic Karen Benevolent Army (DKBA-5)',
    'Union Peace Group':'Union level peace-making group',
    'NSCN':'National Socialist Council of Nagaland (NSCN)',
    'PPLO':'Pa-O Peoples Liberation Organization (PPLO)',
    'PNLO':'Pa-O National Liberation Organisation (PNLO)',
    'UNODC':'United Nations Office on Drugs and Crime (UNODC)',
    'UPWC':'Union Peacemaking Working Committee (UPWC)',
    'NMSP':'New Mon State Party (NMSP)',
    'UEC':'Union Election Commission (UEC)',
    'KIO':'Kachin Independence Organisation (KIO)',
    'UNFC':'United Nationalities Federal Council (UNFC)',
    'NBF': 'Nationalities Brotherhood Federation (NBF)'
    
}

There are spaces/unncessary characters included in the results. This dictionary below corrects persons identified names.

In [6]:
corrected_person_dict={
    'General Aung San':[' Aung San'],
    'Lt. Gen. Ya Pyit':[' Gen. Ya Pyit'],
    'U Khin Maung Soe': [' U Khin Maung Soe  Union'],
    'Lt. Gen. Ye Aung':[' Gen. Ye Aung  Chief'],
    'Maj. Gen. Tun Tun Naung':[' Tun Tun Naung',],
    'Maj. Gen. Aung Kyaw Zaw': [' Aung Kyaw Zaw',],
    'U Myint Soe':[' U Myint Soe  Retired Lt'],
    'Maj. Gen. Aung Soe':[' Aung Soe',],
    'Col. Wun Na Aung':[' Ministry Interior'],
    'Dr. Min Zaw Oo':[' Min Zaw'],
    'Maj. Gen. Saw Isaac Poe':[ ' Gen. Saw Isaac Poe  Quartermaster General',],
    'Maj. Gen. Saw Nay Soe Mya':[' Gen. Saw Nay Soe Mya  Adjutant General',],
    'Dr. Shwe Kah':[ ' Shwe Kah',],
    'Col. Saw Kyaw Than Htay':[' Saw Kyaw Htay'],
    'Lt. Col. Khun Aung Mang':[' Khun Aung Mang  PNLO',],
    'Salai Yaw Aung':[' Salai Yaw Aung  ABSDF'],
    'Col. Hsaung Han':[' Hsaung Han',],
    'Sai Liang':[' Sai Liang '],
    'U Aung Min':[' U Aung Min'],
    'Khu Oo Reh':[' KNPP Khu Oo Reh',],
     'U Saw Mu Shay': [' Kalo Htoo Baw U Saw Mu Shay',],
    'Col. Aung Lwin':[' Aung Lwin', ],
    'Maj. Gen Saw Lar Pwe':[' Gen Saw Lar Pwe',],
    'U Aung Thaung':[' U Aung Thaung',],
    'U Thein Zaw':[' U Thein Zaw',],
    'U Ohn Myint':[' U Ohn Myint'],
    'U Win Tun':[' U Win Tun',],
    'U Sao Aung Myat':[' U Sao Aung Myat'],
    'U Maung Maung Thein':[' U Maung Maung Thein'],
    'Maj-Gen Aung Kyaw Zaw':[' North - East Command Maj - Gen Aung Kyaw Zaw'],
    'Maj-Gen Mya Tun Oo':[' Central East Command'],
    'Maj-Gen Zaw Win':[' Maj - Gen Zaw Win'],
    'U Soe Tint':[' U Soe Tint',],
    'Col Aung Thu.':[' Aung Thu',],
    'General Sao Khay Tai':[' Sao Khay Tai',],
    'Sao Khin Min':[ ' Sao Khin Min',],
    'Sao Khun Hsai':[' Sao Khun Hsai'],
    'Sao Khay Maung':[' Sao Khay Maung'],
    'Sao Kyaw Hla':[' Sao Kyaw Hla'],
    'Sao Htin Mein':[' Sao Htin Mein'],
    'Sao Laung Khay':[' Sao Laung Khay'],
    'Sao Naw Leik':['  Sao Naw Leik'],
    'General Secretary Naw Zipporah Sein':[' Naw Zipporah Sein',],
    'U Saw Lah Pwe':[ ' U Saw Lah Pwe',],
    'U Khun Okkar':[' Khun Okkar'],
    'U Khun Myint Htun': [' U Khun Myint Htun'],
    'U Khin Mg Soe':[' Khin Mg Soe'],
    'U Twan Zaw':[' U Twan Zaw'],
    'U Tun Mrat Naing':[' U Tun Mrat Naing'],
    'Pu Zin Cun':[' Pu Zin Cun Commander',],
    'Gen. Saw Mo Shay':[' Saw Mo Shay',],
    'Koon Abel Tweed':[' Koon Abel Tweed',],
    'Col. Saw Kyaw Dah':[' Saw Kyaw Dah',],
    'Kya Khunsa':[' Kya Khunsa',],
    'Dr. Tin Myo Win':[' Tin Myo Win  Representative',],
    'Dr. May Win Myint':[' Win Myint  Representative',],
    'U Thu Wai':[' U Thu Wai  Representative'],
    'U Kyaw Win':[' Kyaw Win  Relevant Representative'],
    'U Zein Khan Ram':[' Ethnic Armed Organization Group  U Zein Khan Ram  Ethnic Representative Political Parties Group'],
    'U Thein Sein':[' Union Myanmar U Thein Sein'],
    'Pu Zing Cung':[' Pu Zing Cung'],
    'Dr. Sui Khar':[' Sui Khar'],
    'U Win Htun':[' U Win Htun',],
    'U Hung Ngai':[' U Hung Ngai',],
    'Htein Lin':[' Htein Lin']
}
    

It is also useful to know what organisation these people are affiliated with, so a person -> organisation dictionary was made:

In [7]:
person_affilitation={
    'Government of the Republic of the Union of Myanmar':[' Htein Lin',' U Win Htun',' Union Myanmar U Thein Sein',' Kyaw Win  Relevant Representative',' Win Myint  Representative',' Tin Myo Win  Representative',' Aung San', ' Gen. Ya Pyit', ' U Khin Maung Soe  Union', ' Gen. Ye Aung  Chief',' Tun Tun Naung', ' Aung Kyaw Zaw', ' U Myint Soe  Retired Lt', ' Aung Soe', ' Ministry Interior', ' Min Zaw', ' U Aung Min', ' Gen Saw Lar Pwe', ' U Aung Thaung', ' U Thein Zaw', ' U Ohn Myint', ' U Win Tun', ' U Maung Maung Thein', ' North - East Command Maj - Gen Aung Kyaw Zaw',' Central East Command',' Maj - Gen Zaw Win', ],
    'KNU':[' Saw Kyaw Dah',' Gen. Saw Isaac Poe  Quartermaster General', ' Gen. Saw Nay Soe Mya  Adjutant General', ' Naw Zipporah Sein',],
    'CNF':[' Shwe Kah', ' Pu Zin Cun Commander',' Pu Zing Cung', ' Sui Khar'],
    'DKBA':[' Saw Kyaw Htay'],
    'PNLO':[' Khun Aung Mang  PNLO', ' Khun Okkar'],
    'ABSDF':[' Salai Yaw Aung  ABSDF'],
    'RCSS':[' Hsaung Han',' Sai Liang '],
    'KNPP':[' KNPP Khu Oo Reh', ' Koon Abel Tweed',],
    'DKBA':[' Saw Mo Shay',],
    'DKBA-5':[' Kalo Htoo Baw', ' Kaloh Htoo Baw', ' Kalo Htoo Baw U Saw Mu Shay', ' U Saw Lah Pwe'],
    'Karen State Government':[' Aung Lwin',],
    'Shan State Government':[' U Sao Aung Myat', ' Aung Thu',],
    'Chin State Government':[' U Hung Ngai',],
    'SSPP/SSA':[ ' Sao Khay Tai',  ' Sao Khin Min', ' Sao Khun Hsai', ' Sao Khay Maung',' Sao Kyaw Hla',' Sao Htin Mein',' Sao Laung Khay','  Sao Naw Leik'],
    'UPWC':[' Khin Mg Soe'],
    'Arakan National Council':[' U Twan Zaw'],
    'Arakan Army':[' U Tun Mrat Naing'],
    'Lahu Democratic Union':[' Kya Khunsa'],
    'Political Parties Group':[' U Thu Wai  Representative', ' Ethnic Armed Organization Group  U Zein Khan Ram  Ethnic Representative Political Parties Group'],
}

Need to label the results using these dictionaries by applying a function:

In [8]:
def label_org(org_name):
    for key, value in org_dict.items():
        if org_name in value:
            return key    

In [9]:
def match_shortened_to_key(short_name):
    for key, value in shortened_name_dict.items():
        if short_name in value:
            return key

In [10]:
results['corrected_org_named']=results['entity'].apply(label_org)

In [11]:
#results['corrected_org_abr']=results['corrected_org_named'].apply(match_shortened_to_key)

In [12]:
def correct_org(s):
    if (s['corrected_org_named'] == None):
        return (s['entity'])
    else:
        return (s['corrected_org_named'])

In [13]:
results['corrected_entity_1']=results.apply(correct_org, axis=1)

In [14]:
results['corrected_abr_1']=results['corrected_entity_1'].apply(match_shortened_to_key)

In [15]:
def correct_label_org(s):
    if (s['corrected_org_named'] != None):
        return 'ORG'
    else:
        return (s['entity_type'])

In [16]:
results['corrected_entity_type_1']=results.apply(correct_label_org, axis=1)

In [17]:
def match_corrected_label_to_key(corrected_label_name):
    for key, value in corrected_person_dict.items():
        if corrected_label_name in value:
            return key

In [18]:
results['corrected_person_named']=results['corrected_entity_1'].apply(match_corrected_label_to_key)

In [19]:
def match_affiliation_to_key(affiliation_name):
    for key, value in person_affilitation.items():
        if affiliation_name in value:
            return key

In [20]:
results['corrected_person_affiliation']=results['entity'].apply(match_affiliation_to_key)

In [21]:
def correct_label_per(s):
    if (s['corrected_person_named'] != None):
        return 'PERSON'
    else:
        return (s['corrected_entity_type_1'])

In [22]:
results['corrected_entity_type_2']=results.apply(correct_label_per, axis=1)

In [23]:
def correct_entity(s):
    if (s['corrected_person_named']== None):
        return (s['corrected_entity_1'])
    else:
        return (s['corrected_person_named'])

In [24]:
results['corrected_entity']=results.apply(correct_entity, axis=1)

In [25]:
def affil(s):
    if (s['corrected_person_affiliation']==None):
        return (s['corrected_entity_1'])
    else:
        return (s['corrected_person_affiliation'])

In [26]:
results['affiliated_org']=results.apply(affil, axis=1)

In [27]:
results['affiliated_org_abr']=results['affiliated_org'].apply(match_shortened_to_key)

In [28]:
cols=['doc_id',
 'agreement_id',
 'sentence_number','corrected_entity_type_2',
 'corrected_entity',
 'affiliated_org',
 'affiliated_org_abr']

In [29]:
r=results[cols]

In [30]:
r1=r.rename(columns={'corrected_entity_type_2':'entity_type',
 'corrected_entity':'entity_named',
 'affiliated_org':'entity_affiliated_org',
 'affiliated_org_abr':'entity_affiliated_org_abr'})

In [31]:
r1

,doc_id,agreement_id,sentence_number,entity_type,entity_named,entity_affiliated_org,entity_affiliated_org_abr
0,0,1321,0,ORG,Chin National Front (CNF),Chin National Front (CNF),CNF
1,0,1321,0,ORG,Chinland Guardian,Chinland Guardian,None
2,0,1321,0,ORG,Chin National Front (CNF),Chin National Front (CNF),CNF
3,0,1321,0,ORG,Peace Committee Burma,Peace Committee Burma,None
4,0,1321,0,GPE,Rangoon,Rangoon,None
...,...,...,...,...,...,...,...
1372,35,776,4,ORG,Rakhine State Peace Discussion Group,Rakhine State Peace Discussion Group,None
1373,35,776,4,ORG,State Security,State Security,None
1374,35,776,4,PERSON,Htein Lin,Government of the Republic of the Union of Mya...,Gov. of Myanmar
1375,35,776,5,ORG,Rakhine State Liberation Party (RSLP),Rakhine State Liberation Party (RSLP),RSLP


In [32]:
r1.to_csv('Myanmar_corrected_results_full.csv')

Get dataframe of just organisations and people (without location or gpe entities):

In [33]:
list(r)

['doc_id',
 'agreement_id',
 'sentence_number',
 'corrected_entity_type_2',
 'corrected_entity',
 'affiliated_org',
 'affiliated_org_abr']

In [34]:
r_org=pd.DataFrame(r1.loc[r1['entity_type']=='ORG'])

In [35]:
r_per=pd.DataFrame(r1.loc[r1['entity_type']=='PERSON'])

In [36]:
r_org.to_csv('Myanmar_entities_orgs.csv')

In [37]:
r_per.to_csv('Myanmar_persons_orgs.csv')

In [38]:
df=pd.concat([r_org, r_per])

In [39]:
df.to_csv('Myanmar_entities_orgs_persons.csv')

Some entities were not recognised by the spaCy model, so a manual list of people was created:

In [40]:
manual=pd.read_csv('persons_not_identified.csv')

In [41]:
manual['entity_type']='PERSON'

In [42]:
df['NER']='SpaCy'

In [43]:
df

,doc_id,agreement_id,sentence_number,entity_type,entity_named,entity_affiliated_org,entity_affiliated_org_abr,NER
0,0,1321,0,ORG,Chin National Front (CNF),Chin National Front (CNF),CNF,SpaCy
2,0,1321,0,ORG,Chin National Front (CNF),Chin National Front (CNF),CNF,SpaCy
5,0,1321,1,ORG,Chin National Front (CNF),Chin National Front (CNF),CNF,SpaCy
9,0,1321,1,ORG,Chin National Front (CNF),Chin National Front (CNF),CNF,SpaCy
10,0,1321,1,ORG,Union Peace Dialogue Joint Committee (UPDJC),Union Peace Dialogue Joint Committee (UPDJC),UPDJC,SpaCy
...,...,...,...,...,...,...,...,...
1272,31,771,0,PERSON,Dr. Sui Khar,CNF,CNF,SpaCy
1327,31,771,16,PERSON,U Aung Min,Government of the Republic of the Union of Mya...,Gov. of Myanmar,SpaCy
1329,31,771,16,PERSON,U Win Htun,Government of the Republic of the Union of Mya...,Gov. of Myanmar,SpaCy
1330,31,771,16,PERSON,U Hung Ngai,Chin State Government,None,SpaCy


In [44]:
manual['person_affiliation']=manual['person_affiliation'].replace('Gov', 'Gov. of Myanmar')

In [46]:
m=manual.drop(columns=['Unnamed: 0', 'per_correction']).rename(columns={'entity':'entity_named', 'person_affiliation':'entity_affiliated_org_abr'})

Merge this with people and organisations:

In [47]:
both=pd.concat([df, m])

In [48]:
both.to_csv('Myanmar_entities_person_orgs_withManual.csv')

## ICEWS Dictionaries

In [49]:
icews=pd.read_csv('icews.actors.20181119.csv')

In [51]:
myanmar=pd.DataFrame(icews.loc[icews['Country']=='Myanmar'])

In [69]:
myanmar

,Country,Actor Name,Actor Type,Affiliation To,Affiliation Start Date,Affiliation End Date,Aliases
64958,Myanmar,88 Generation Students,Group,Education (Myanmar),beginning of time,end of time,88 Generation Students
64959,Myanmar,88 Generation Students,Group,Protestors / Popular Opposition / Mobs (Myanmar),beginning of time,end of time,88 Generation Students
64960,Myanmar,AKYSU,Group,Education (Myanmar),beginning of time,end of time,All Kachin Students Union || AKYSU
64961,Myanmar,All Burma Democratic Force,Group,Protestors / Popular Opposition / Mobs (Myanmar),beginning of time,end of time,ABDF || All Burma Democratic Force
64962,Myanmar,All Burma Federation of Student Unions,Group,Education (Myanmar),beginning of time,end of time,All Burma Federation Of Student Unions || ABFSU
...,...,...,...,...,...,...,...
65438,Myanmar,Zaw Min,Individual,Army (Myanmar),beginning of time,28/08/2012,Zaw Min
65439,Myanmar,Zaw Min,Individual,Elite (Myanmar),beginning of time,end of time,Zaw Min
65440,Myanmar,Zaw Min,Individual,Energy Ministry (Myanmar),30/03/2011,28/08/2012,Zaw Min
65441,Myanmar,Zaw Min,Individual,Pyithu Hluttaw,31/01/2011,30/03/2011,Zaw Min


In [79]:
myanmar.to_csv('ICEWS_Myanmar_actors.csv')

In [78]:
#myanmar_aliases=myanmar['Aliases'].str.split('|', expand=True)